<a href="https://colab.research.google.com/github/rosafilgueira/PythonCloneDetection/blob/main/Vector_Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Dependencies

In [28]:
 %pip install transformers scikit-learn
 !pip install -U sentence-transformers
 !python --version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Python 3.8.10


In [3]:
 %pip install transformers datasets tqdm scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import ast
import astunparse
from scipy.optimize import linesearch

import torch
import os
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
import inspect
import ast
import astunparse


# function to extract source code lines of each function 
def extract_function_source_code(file_name):
    func=[]
    with open(file_name, 'r') as file:
        source_code = file.read() 
        parsed_code = ast.parse(source_code)
        function_list = [node for node in ast.walk(parsed_code) if isinstance(node, ast.FunctionDef)]
        for f_obj in function_list:
             data=astunparse.unparse(f_obj)
             func.append(data)
        return func

def create_repo_embeddings(repo_path):
  tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
  model = AutoModel.from_pretrained("microsoft/graphcodebert-base")
  
 
 # Initialize the list to store the file embeddings
  # Iterate through all the source files in the repository

  files_embedding=[]
  for file in os.listdir(repo_path):
    f_embedding=[]
    # Extract the source code from each file
    functions = extract_function_source_code(os.path.join(repo_path, file))
    for f_code in functions: 
      #print(f_code)
      # Generate the embedding for each function
      code_tokens=tokenizer.tokenize(f_code)
      #print(code_tokens)
      tokens=[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
      #print(tokens)
      tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
      #print(tokens_ids)
      embedding=model(torch.tensor(tokens_ids)[None,:])[0][0,0]
      # Append the embedding to the list
      f_embedding.append(embedding)
    if len(f_embedding) > 0:
        f = torch.mean(torch.stack(f_embedding), dim=0)
        files_embedding.append(f)
     
  # Aggregate the file embeddings to get the repo code embedding
  repo_embedding = torch.mean(torch.stack(files_embedding), dim=0)
  return repo_embedding

repo1_embedding=create_repo_embeddings("./repo")
repo2_embedding=create_repo_embeddings("./repo2")
repo3_embedding=create_repo_embeddings("./repo3")
repo4_embedding=create_repo_embeddings("./repo4")

#cos = torch.nn.CosineSimilarity(dim=0)

output = nn.CosineSimilarity(dim=0, eps=1e-6)(repo1_embedding, repo2_embedding)
print("Cosine Similarity between repo1 and repo2: %s" % output)


output_2 = nn.CosineSimilarity(dim=0, eps=1e-6)(repo1_embedding, repo3_embedding)
print("Cosine Similarity between repo1 and repo3: %s" % output_2)

output_3 = nn.CosineSimilarity(dim=0, eps=1e-6)(repo2_embedding, repo4_embedding)
print("Cosine Similarity between repo2 and repo4: %s" % output_3)

#output = cos(repo1_embedding, repo2_embedding)
#print("Cosine Similarity between repo1 and repo2: %s" % output)
#output2 = cos(repo1_embedding, repo3_embedding)
#print("Cosine Similarity between repo1 and repo3: %s" % output2)
#output3 = cos(repo4_embedding, repo3_embedding)
#print("Cosine Similarity between repo4 and repo3: %s" %output3)


Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

Cosine Similarity between repo1 and repo2: tensor(1.0000, grad_fn=<SumBackward1>)
Cosine Similarity between repo1 and repo3: tensor(0.9037, grad_fn=<SumBackward1>)
Cosine Similarity between repo2 and repo4: tensor(0.9292, grad_fn=<SumBackward1>)


In [16]:
! rm -rf ./repo1/.ipynb_checkpoints


In [13]:
import ast
import astunparse
from scipy.optimize import linesearch
import numpy as np

import torch
import os
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
import inspect
import ast
import astunparse
from sklearn.decomposition import PCA
import torch.nn.functional as F


# function to extract source code lines of each function 
def extract_function_source_code(file_name):
    func=[]
    with open(file_name, 'r') as file:
        source_code = file.read() 
        parsed_code = ast.parse(source_code)
        function_list = [node for node in ast.walk(parsed_code) if isinstance(node, ast.FunctionDef)]
        for f_obj in function_list:
             data=astunparse.unparse(f_obj)
             func.append(data)
        return func

def create_repo_embeddings(repo_path):
  tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
  model = AutoModel.from_pretrained("microsoft/graphcodebert-base")
  
 
 # Initialize the list to store the file embeddings
  # Iterate through all the source files in the repository

  files_embedding=[]
  for file in os.listdir(repo_path):
    f_embedding=[]
    # Extract the source code from each file
    functions = extract_function_source_code(os.path.join(repo_path, file))
    for f_code in functions: 
      #print(f_code)
      # Generate the embedding for each function
      code_tokens=tokenizer.tokenize(f_code, max_length=50,padding='max_length', truncation=True)
      #print(code_tokens)
      tokens=[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
      #print(tokens)
      tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
      #print(tokens_ids)
      embedding=model(torch.tensor(tokens_ids)[None,:])[0][0,0]
      #print("the embedding shape")
      #print(embedding.shape)
      f_embedding.append(embedding)
    if len(f_embedding) > 0:
        f= torch.stack(f_embedding, dim=0)
        print("---the f-stack shape BEFORE")
        print(f.shape)
        files_embedding.append(f)
        #print(files_embedding.shape)
     
  # Aggregate the file embeddings to get the repo code embedding
  repo_embedding = torch.cat(files_embedding, dim=0) 
  print("--- the repo-stack shape BEFORE PCA--")
  print(repo_embedding.shape)

  #repo_resize = repo_embedding.view(40, 768)
  #repo_resize = F.pad(input=repo_embedding, pad=(0, 0), mode='constant', value=0)
  #repo_resize = nn.ConstantPad1d((40 - repo_embedding.size()[0],0) ,0 )(repo_embedding)
  
  padder = torch.zeros(40 - repo_embedding.size()[0], 768)
  repo_resize = torch.cat([repo_embedding,padder], dim = 0) 

  print(repo_resize)
  print("--- the repo-resize shape BEFORE PCA--")
  print(repo_resize.shape)
  repo_dim=repo_resize.size()

  num_comp=min(repo_dim[0],repo_dim[1])
  print("Num_comp %s" % num_comp)

  # Append the embedding to the list
  # f_embedding.append(pca_embedding)
  X1 = repo_resize.cpu().detach().numpy()
  pca = PCA(n_components = num_comp)
  X1 = pca.fit_transform(X1)
  repo_pca = torch.from_numpy(X1)

  print("the final repo shape AFTER PCA--")
  print(repo_pca.shape)
  return repo_pca

repo1_embedding=create_repo_embeddings("./repo")
repo2_embedding=create_repo_embeddings("./repo1")


cos = torch.nn.CosineSimilarity(dim=1)
output = cos(repo1_embedding, repo2_embedding)
print("Cosine Similarity between repo1 and repo2: %s" % output)



Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

---the f-stack shape BEFORE
torch.Size([2, 768])
---the f-stack shape BEFORE
torch.Size([1, 768])
---the f-stack shape BEFORE
torch.Size([1, 768])
--- the repo-stack shape BEFORE PCA--
torch.Size([4, 768])
tensor([[ 0.1580, -0.6669,  0.1874,  ...,  0.1207, -0.1431,  0.6867],
        [ 0.0469, -0.3738,  0.1185,  ...,  0.0479,  0.0517,  0.2263],
        [ 0.0652, -0.4650, -0.0373,  ..., -0.0201, -0.0497,  0.4484],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<CatBackward0>)
--- the repo-resize shape BEFORE PCA--
torch.Size([40, 768])
Num_comp 40
the final repo shape AFTER PCA--
torch.Size([40, 40])


Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

---the f-stack shape BEFORE
torch.Size([10, 768])
---the f-stack shape BEFORE
torch.Size([6, 768])
---the f-stack shape BEFORE
torch.Size([4, 768])
---the f-stack shape BEFORE
torch.Size([4, 768])
---the f-stack shape BEFORE
torch.Size([3, 768])
---the f-stack shape BEFORE
torch.Size([7, 768])
---the f-stack shape BEFORE
torch.Size([6, 768])
--- the repo-stack shape BEFORE PCA--
torch.Size([40, 768])
tensor([[ 0.1336, -0.7445,  0.3154,  ..., -0.0855, -0.1292,  0.3474],
        [ 0.0901, -0.3895,  0.1428,  ..., -0.0715,  0.0547,  0.2567],
        [ 0.1244, -0.3469,  0.1493,  ..., -0.0640,  0.0600,  0.3892],
        ...,
        [ 0.0700, -0.4424,  0.0748,  ..., -0.0348, -0.0222,  0.2199],
        [ 0.1304, -0.0683,  0.0717,  ..., -0.5634, -0.4308,  0.2660],
        [ 0.0640, -0.4560,  0.0659,  ..., -0.0450, -0.0067,  0.2299]],
       grad_fn=<CatBackward0>)
--- the repo-resize shape BEFORE PCA--
torch.Size([40, 768])
Num_comp 40
the final repo shape AFTER PCA--
torch.Size([40, 40])
Cosi

In [ ]:
 print("--- the repo-stack shape BEFORE PCA--")
  print(repo_embedding.shape)
  repo_dim=repo_embedding.size()
  num_comp=min(repo_dim[0],repo_dim[1])

  # Append the embedding to the list
  # f_embedding.append(pca_embedding)
  X1 = repo_embedding.cpu().detach().numpy()
  pca = PCA(n_components = num_comp)
  X1 = pca.fit_transform(X1)
  repo_pca = torch.from_numpy(X1)

  print("the final repo shape AFTER PCA--")
  print(repo_pca.shape)
  return repo_embedding

In [7]:
import torch.nn.functional as F

# cosine similarity = normalize the vectors & multiply
C = F.normalize(repo1_embedding) @ F.normalize(repo2_embedding).t()
print(C.shape)
print(repo1_embedding.shape)
print(repo2_embedding.shape)

a_norm = torch.nn.functional.normalize(repo1_embedding, p=2, dim=1)
b_norm = torch.nn.functional.normalize(repo2_embedding, p=2, dim=1)
d= torch.mm(a_norm, b_norm.transpose(0, 1))
print(d)

torch.Size([4, 40])
torch.Size([4, 768])
torch.Size([40, 768])
tensor([[0.9402, 0.9189, 0.9299, 0.9230, 0.9689, 0.9012, 0.9639, 0.9316, 0.7989,
         0.4440, 0.9402, 0.9189, 0.9299, 0.9230, 0.9689, 0.9012, 0.9402, 0.9189,
         0.9299, 0.9230, 0.9402, 0.9189, 0.9299, 0.9424, 0.9150, 0.7296, 0.9092,
         0.9402, 0.9189, 0.9230, 0.9308, 0.9685, 0.9086, 0.9245, 0.4014, 0.9545,
         0.4058, 0.9161, 0.4100, 0.9148],
        [0.8716, 0.9897, 0.9817, 0.9822, 0.9489, 0.8130, 0.9227, 0.9814, 0.7090,
         0.4386, 0.8716, 0.9897, 0.9817, 0.9822, 0.9489, 0.8130, 0.8716, 0.9897,
         0.9817, 0.9822, 0.8716, 0.9897, 0.9817, 0.9730, 0.9829, 0.6504, 0.9893,
         0.8716, 0.9897, 0.9822, 0.9800, 0.9340, 0.9927, 0.9893, 0.3804, 0.9465,
         0.4114, 0.9854, 0.4137, 0.9858],
        [0.8815, 0.9797, 0.9848, 0.9805, 0.9628, 0.8210, 0.9343, 0.9826, 0.7187,
         0.4322, 0.8815, 0.9797, 0.9848, 0.9805, 0.9628, 0.8210, 0.8815, 0.9797,
         0.9848, 0.9805, 0.8815, 0.9797, 0.